In [25]:
import os
current_dir = os.getcwd()

# Define the Delta table path relative to the current directory
delta_table_path = os.path.join(current_dir, "Delta_Lake/")
folder_path = 'MovieLens_32M'
delta_table_movielens_path = delta_table_path+'MovieLens/'
folder_path = f'IMDB_datasets'
delta_table_imdb_path = f'{delta_table_path}IMDB/'

In [8]:
import os

print(os.getenv('SPARK_HOME'))
print(os.getenv('HADOOP_HOME'))
print(os.getenv('JAVA_HOME'))


C:\Program Files\Spark
C:\Program Files\hadoop-3.0.0
C:\Program Files\Java\jdk-21


Each .parquet file represents a partition of data (in case of partitioned data) or a set of rows for unpartitioned data.

.CRC files are automatically generated by Delta Lake during file writes and are used for quick integrity checks during reads.

In [2]:
import pyspark
from delta import *

builder = pyspark.sql.SparkSession.builder.appName("LocalDeltaTable") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

# MovieLens

In [ ]:

for filename in os.listdir(folder_path):
    # Check if the file is a CSV
    if filename.endswith('.csv'):
        # Construct full file path
        file_path = os.path.join(folder_path, filename)
        
        try:
            df = spark.read.format('csv').option('header', True).load(file_path)
            # df.show()

            df.write.format('delta').save(delta_table_movielens_path+filename.removesuffix('csv'))
            
            
        except Exception as e:
            print(f"Error reading {filename}: {e}")

In [ ]:
# spark.read.format('delta').load(delta_table_movielens_path+'links').show()

# IMDB

In [ ]:


for filename in os.listdir(folder_path):
    # Check if the file is a CSV
    if filename.endswith('.tsv'):
        # Construct full file path
        file_path = os.path.join(folder_path, filename)
        print(file_path)
        try:
            df = spark.read.format('csv').option('header', True).option('delimiter', '\t').load(file_path)
            # # df.show()

            df.write.format('delta').save(delta_table_imdb_path+filename.removesuffix('tsv'))
            
            
        except Exception as e:
            print(f"Error reading {filename}: {e}")

IMDB_datasets\name.basics.tsv
IMDB_datasets\title.akas.tsv
IMDB_datasets\title.basics.tsv
IMDB_datasets\title.crew.tsv
IMDB_datasets\title.episode.tsv
IMDB_datasets\title.principals.tsv
IMDB_datasets\title.ratings.tsv


In [27]:
def getSpecificDayTable(delta_path, date):
    history = spark.sql(f"DESCRIBE HISTORY delta.`{delta_path}`").toPandas()
    timestamp = history[history["timestamp"].dt.strftime("%Y-%m-%d") == date]
    return str(timestamp.loc[0, 'timestamp'])

In [31]:
from datetime import datetime
time = getSpecificDayTable(delta_table_imdb_path+'title.akas', datetime.today().strftime('%Y-%m-%d'))
spark.read.format('delta').option('timestampAsOf', time).load(delta_table_imdb_path+'title.akas').show()

+----------+--------+---------------------------+------+--------+--------+----------+---------------+
|   titleId|ordering|                      title|region|language|   types|attributes|isOriginalTitle|
+----------+--------+---------------------------+------+--------+--------+----------+---------------+
|tt15575488|       6|       Folge vom 13. Okt...|    DE|      de|      \N|        \N|              0|
|tt15575488|       7|2021年10月13日 のエピソード|    JP|      ja|      \N|        \N|              0|
|tt15575488|       8|       Episodio fechado ...|    ES|      es|      \N|        \N|              0|
| tt1557549|       1|              Dare to Dream|    \N|      \N|original|        \N|              1|
| tt1557549|       2|              Dare to Dream|    CA|      \N|      \N|        \N|              0|
|tt15575490|       1|       Episode dated 14 ...|    \N|      \N|original|        \N|              1|
|tt15575490|       2|       Episódio datado d...|    PT|      pt|      \N|        \N|      